<a href="https://colab.research.google.com/github/nomomon/anime-recommendations/blob/main/User_Anime_Hybrid_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Kaggle API
username = str(input("username: "))
key = str(input("key: "))

f = open("kaggle.json", "w")
f.write('{"username":"'+username+'","key":"'+key+'"}')
f.close()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

import os, re
from zipfile import ZipFile

with ZipFile('/content/anime-recommendation-database-2020.zip', 'r') as zipObj:
   zipObj.extractall()

def purge(dir, pattern):
    for f in os.listdir(dir):
        if pattern in f:
            os.remove(os.path.join(dir, f))

purge("/content/", ".zip")
purge("/content/", ".json")

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

In [2]:
ratings = pd.read_csv("/content/animelist.csv")

In [3]:
ratings.head(5)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [4]:
ratings["rating"].value_counts()

0     46827035
8     15422150
7     14244633
9     10235934
6      7543377
10     7144392
5      4029645
4      1845854
3       905700
2       545339
1       480688
Name: rating, dtype: int64

In the dataset, if a person hasn't left a rating to an anime, it was marked as a zero. Let's remove the zeros, as they will just introduce noise later on. Might as well remove the `watched_episodes`.

In [5]:
ratings = ratings[ratings["rating"] > 0]
ratings = ratings.drop(columns = ["watched_episodes"])

In [6]:
num_users = ratings["user_id"].nunique()
num_anime = ratings["anime_id"].nunique()

print(f"There are a total of {num_users} users found.")
print(f"There are a total of {num_anime} anime found.")

print("Max user id:", ratings["user_id"].max())
print("Max anime id:", ratings["anime_id"].max())

There are a total of 313670 users found.
There are a total of 17172 anime found.
Max user id: 353404
Max anime id: 48456


In [7]:
def transform(df):
    df.loc[df["watching_status"] != 4, ["watching_status"]] = 1  # Not Dropped
    df.loc[df["watching_status"] == 4, ["watching_status"]] = 0  # Dropped

    df.rename(columns = {'watching_status': 'interaction'}, inplace = True)

In [8]:
transform(ratings)

In [9]:
ratings.head()

,user_id,anime_id,rating,interaction
0,0,67,9,1
1,0,6702,7,1
2,0,242,10,1
4,0,21,10,1
5,0,24,9,1


If we take a look at the anime and user ids, we can notice that some values there are missing. Let's make a two helper functions, for easy conversion between dataset ids and embedding ids.

In [10]:
embId2user = sorted(ratings["user_id"].unique())
embId2anime = sorted(ratings["anime_id"].unique())

In [11]:
user2embId = {v: k for k, v in enumerate(embId2user)}
anime2embId = {v: k for k, v in enumerate(embId2anime)}

Now, let's make the dataset. It will look like a quadruples of `(user_id, anime_embId, rating, interaction)`. We'll make 3 sets: `train`, `test` and `val`.

In [12]:
np.random.seed(seed = 42)

def makeDataSet(df, split=0.95):
    n = df.to_numpy()
    
    n = np.random.permutation(n)

    x = n[:, :2]
    y = n[:, 2:]

    for i in range(x.shape[0]):
        x[i][0] = user2embId[x[i][0]]
        x[i][1] = anime2embId[x[i][1]]

    s1 = int(split * n.shape[0])
    s2 = s1 + int((1 - split) * n.shape[0] / 10)
    
    return (x[:s1], y[:s1], x[s1:s2], y[s1:s2], x[s2:], y[s2:])

In [13]:
x_train, y_train, x_test, y_test, x_val, y_val = makeDataSet(ratings)

In [14]:
y_train[:, 1].mean()

0.968792715171437

# Models

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Neural Network

In [21]:
class HybridNeuralNetworkModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(HybridNeuralNetworkModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')
        
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid')

        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        conc = self.concat([user_embedding, item_embedding])
        x = self.dense1(conc)
        
        x1 = self.dense2(x)
        x2 = self.dense3(x)

        return x1, x2

In [22]:
hnn_model = HybridNeuralNetworkModel(num_users = num_users, 
                                     num_items = num_anime, 
                                     embedding_dim = 64)

hnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = {
        "output_1": tf.keras.losses.MeanSquaredError(),
        "output_2": tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights = {
        "output_1": 1.0,
        "output_2": 5.0
    },
    metrics = [
        tf.keras.metrics.RootMeanSquaredError("RMSE")
    ],
    run_eagerly = True
)

In [23]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = hnn_model.fit(x = x_train, 
                        y = {
                            "output_1": y_train[:, 0],
                            "output_2": y_train[:, 1]
                        },
                        batch_size = 64, 
                        epochs = 100, 
                        steps_per_epoch = 1000,
                        callbacks = [callback],
                        # validation_data = (x_test, y_test),
                        # validation_steps = 10,
                        # validation_batch_size = 64
                    )

Epoch 1/100
1000/1000 [==============================] - 38s 36ms/step - loss: 10.9900 - output_1_loss: 10.0567 - output_2_loss: 0.1867 - output_1_RMSE: 3.1712 - output_2_RMSE: 0.2163
Epoch 2/100
1000/1000 [==============================] - 35s 35ms/step - loss: 3.3543 - output_1_loss: 2.6759 - output_2_loss: 0.1357 - output_1_RMSE: 1.6358 - output_2_RMSE: 0.1731
Epoch 3/100
1000/1000 [==============================] - 36s 36ms/step - loss: 3.1916 - output_1_loss: 2.5294 - output_2_loss: 0.1324 - output_1_RMSE: 1.5904 - output_2_RMSE: 0.1711
Epoch 4/100
1000/1000 [==============================] - 36s 36ms/step - loss: 3.1645 - output_1_loss: 2.5061 - output_2_loss: 0.1317 - output_1_RMSE: 1.5831 - output_2_RMSE: 0.1714
Epoch 5/100
1000/1000 [==============================] - 36s 36ms/step - loss: 3.1044 - output_1_loss: 2.4361 - output_2_loss: 0.1337 - output_1_RMSE: 1.5608 - output_2_RMSE: 0.1732
Epoch 6/100
1000/1000 [==============================] - 36s 36ms/step - loss: 3.0168 - 

In [26]:
nn_model.predict([[1, 1]])

(array([[8.27286]], dtype=float32), array([[0.9951828]], dtype=float32))

In [25]:
# hnn_model.save_weights(
#     "/content/drive/MyDrive/AnimeRecommendations/Hybrid/NeuralNetworkModel/model", overwrite=True
# )

In [ ]:
hnn_model.load_weights("/content/drive/MyDrive/AnimeRecommendations/Hybrid/NeuralNetworkModel/model")

## Comparing the models

In [27]:
hnn_model.evaluate(x = x_val, y = [y_val[:, 0], y_val[:, 1]])

87747/87747 [==============================] - 1101s 13ms/step - loss: 2.5118 - output_1_loss: 1.9339 - output_2_loss: 0.1156 - output_1_RMSE: 1.3906 - output_2_RMSE: 0.1664


[2.5118048191070557,
 1.9338595867156982,
 0.11558714509010315,
 1.3906327486038208,
 0.16644959151744843]